In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,StratifiedKFold,RandomizedSearchCV,train_test_split
from sklearn.ensemble import BaggingClassifier



In [20]:
df = pd.read_csv('Iris.csv')

In [21]:
df = df.iloc[:,1:]

In [22]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [25]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
df['Species'] = encode.fit_transform(df['Species'])

In [26]:
X = df.iloc[ : , 0:4]
Y = df.iloc[:,-1]

In [27]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [36]:
X_test,X_train,Y_test,Y_train = train_test_split(X,Y, test_size= 0.2 , random_state= 45)

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced',random_state=45)
param_dist = {
'criterion':['gini','entropy'],
'splitter':['best','random'],
'max_depth':[3,5,7,10,None],
'min_samples_split':[2 ,5 , 8 ,10],
'max_features': [None, 'sqrt', 'log2', 0.5, 0.8]
}

rand = RandomizedSearchCV(dt, param_distributions=param_dist,n_jobs = -1 ,cv = 3,error_score='raise')
rand.fit(X_train,Y_train)
dt1 = rand.best_estimator_




In [347]:
y_pre = dt1.predict(X_test)
accuracy_score(Y_test,y_pre)

0.9666666666666667

In [350]:
stra = StratifiedKFold(n_splits=20)
cross_val_score(dt1,X,Y,cv = stra , n_jobs = -1)

array([1.        , 0.875     , 0.875     , 1.        , 1.        ,
       1.        , 0.875     , 1.        , 1.        , 1.        ,
       0.71428571, 0.85714286, 0.85714286, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [348]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight='balanced', random_state=45)

In [352]:
y_pre = dt.predict(X_test)
accuracy_score(Y_test,y_pre)

0.9416666666666667

In [353]:
stra = StratifiedKFold(n_splits=20)
cross_val_score(dt,X,Y,cv = stra , n_jobs = -1)

array([1.        , 1.        , 0.875     , 1.        , 1.        ,
       1.        , 0.875     , 1.        , 1.        , 0.875     ,
       0.85714286, 0.85714286, 0.71428571, 1.        , 0.85714286,
       1.        , 1.        , 1.        , 1.        , 1.        ])

# Bagging

In [146]:
from sklearn.model_selection import GridSearchCV

In [241]:
param_dist2 = {
'n_estimators':[100,200,300],
'max_samples':[0.7],
'random_state':[45],
'max_features':[0.5],
}


In [324]:
param_dist3 = {
'n_estimators':[100],
'max_samples':[0.5,0.7,.3],
'random_state':[ 45],
'max_features':[0.5],
}

In [ ]:
bag_rf = BaggingClassifier(estimator=dt1,bootstrap=True,oob_score=True)

bagging = GridSearchCV(bag_rf, param_grid=param_dist3,n_jobs = -1 ,cv = 3,error_score='raise',)
bagging.fit(X_train,Y_train)

GridSearchCV(cv=3, error_score='raise',
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                          max_depth=7,
                                                                          min_samples_split=5,
                                                                          random_state=45,
                                                                          splitter='random'),
                                         oob_score=True),
             n_jobs=-1,
             param_grid={'max_features': [0.5], 'max_samples': [0.5, 0.7, 0.3],
                         'n_estimators': [100], 'random_state': [45]})

In [327]:
bagging.best_params_

{'max_features': 0.5,
 'max_samples': 0.7,
 'n_estimators': 100,
 'random_state': 45}

In [329]:
rf =  bagging.best_estimator_

In [330]:
bagging.best_params_
print("Best params:", bagging.best_params_)
print("Best score:", bagging.best_score_)
print("OOB Score:", rf.oob_score_)

Best params: {'max_features': 0.5, 'max_samples': 0.7, 'n_estimators': 100, 'random_state': 45}
Best score: 1.0
OOB Score: 0.9666666666666667


In [331]:
y_pred = rf.predict(X_test)

In [332]:
from sklearn.metrics import accuracy_score,classification_report

In [333]:
classification_report(Y_test,y_pred)

'              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00        39\n           1       0.97      0.91      0.94        43\n           2       0.90      0.97      0.94        38\n\n    accuracy                           0.96       120\n   macro avg       0.96      0.96      0.96       120\nweighted avg       0.96      0.96      0.96       120\n'

In [334]:
y_pred = rf.predict(X_test)
accuracy_score(Y_test,y_pred)


0.9583333333333334

In [343]:
stra = StratifiedKFold(n_splits=20)
cross_val_score(rf,X,Y,cv = stra , n_jobs = -1)

array([1.        , 1.        , 0.875     , 1.        , 1.        ,
       1.        , 0.875     , 1.        , 1.        , 1.        ,
       0.85714286, 0.85714286, 0.71428571, 1.        , 0.85714286,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [344]:
from sklearn.ensemble import RandomForestClassifier
rand = RandomForestClassifier(class_weight='balanced')
rand.fit(X_train,Y_train)


RandomForestClassifier(class_weight='balanced')

In [345]:
y_pred = rand.predict(X_test)
accuracy_score(Y_test,y_pred)

0.9416666666666667

In [340]:
cross_val_score(rand,X,Y,cv = stra , n_jobs = -1)

array([1.        , 1.        , 0.875     , 1.        , 1.        ,
       1.        , 0.875     , 1.        , 1.        , 1.        ,
       0.85714286, 0.85714286, 0.71428571, 1.        , 0.85714286,
       1.        , 1.        , 1.        , 1.        , 1.        ])

# Pasting

In [355]:
bag = BaggingClassifier(
    estimator=dt1,
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose = 1,
    n_jobs=-1
)

In [357]:
bag.fit(X_train,Y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(Y_test,y_pred))

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.1s remaining:    1.7s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.3s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.7s


Pasting classifier 0.9083333333333333


[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.2s finished



# Random Subspaces

In [359]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [360]:
bag.fit(X_train,Y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(Y_test,y_pred))

Random Subspaces classifier 0.9333333333333333


# Random Patches

In [362]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [363]:
bag.fit(X_train,Y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(Y_test,y_pred))

Random Patches classifier 0.925



# Bagging Tips

Bagging generally gives better results than Pasting

Good results come around the 25% to 50% row sampling mark

Random patches and subspaces should be used while dealing with high dimensional data

To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV